In [2]:
%load_ext autoreload
%autoreload 2

env = 'stag'
pk_project = 373987
execute = False

import os
import pandas as pd
import numpy as np
import duckdb
import plotly.express as px

import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.graphs as graphs
import geovpylib.pks as pks
import geovpylib.recordlinkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u

eta = u.Eta()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import Geographical place

## List record linkage

In [4]:
record_linkage = pd.read_csv('../../data/record-linkage-bhp-named-place-geov-geo-places-filled.csv')
record_linkage = record_linkage[record_linkage['doublon'] == 'oui']
record_linkage = record_linkage[['pk_bhp', 'pk_gv']]

a.infos(record_linkage)

Shape:  (598, 2) - extract:


,pk_bhp,pk_gv
0,1,205134
1,3,6201188
2,7,2220550
3,13,1485424
4,19,1876219


## Fetch all named places

In [5]:
named_places = u.read_df('../../data/bhp/named_place.csv')
named_places = named_places[['concat_standard_name', 'standard_latitude', 'standard_longitude']]

In [6]:
named_places

,pk_named_place,concat_napl,concat_standard_name,begin_year,end_year,standard_latitude,standard_longitude,notes,creator,creation_time,modifier,modification_time,fk_abob_type_napl,notes_begin,certainty_begin,notes_end,certainty_end
0,15922,NaPl15922,Vbre,<NA>,<NA>,<NA>,<NA>,Commune existante au 1er janvier 2009. Importe...,4,2009-10-28 16:09:23.000,4,2014-12-03 17:04:13,697,<NA>,<NA>,<NA>,<NA>
1,15923,NaPl15923,Ventenac,<NA>,<NA>,<NA>,<NA>,Commune existante au 1er janvier 2009. Importe...,4,2009-10-28 16:09:23.000,4,2014-12-03 17:04:13,697,<NA>,<NA>,<NA>,<NA>
2,15924,NaPl15924,Verdun,<NA>,<NA>,<NA>,<NA>,Commune existante au 1er janvier 2009. Importe...,4,2009-10-28 16:09:23.000,4,2014-12-03 17:04:13,697,<NA>,<NA>,<NA>,<NA>
3,15925,NaPl15925,Vernajoul,<NA>,<NA>,<NA>,<NA>,Commune existante au 1er janvier 2009. Importe...,4,2009-10-28 16:09:23.000,4,2014-12-03 17:04:13,697,<NA>,<NA>,<NA>,<NA>
4,15926,NaPl15926,Vernaux,<NA>,<NA>,<NA>,<NA>,Commune existante au 1er janvier 2009. Importe...,4,2009-10-28 16:09:23.000,4,2014-12-03 17:04:13,697,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127415,126952,NaPl126952,20_132_18 : parcelle de la zone de Paris,1945,1945,48.877147,2.409766,Parcelles situes au droit de l'ancien bastion ...,75,2016-02-10 11:02:10.420,75,2016-02-11 13:49:21,697,4,1,1,1
127416,126957,NaPl126957,20_132_23 : parcelle de la zone de Paris,1945,1945,48.876642,2.410032,Parcelles situes au droit de l'ancien bastion ...,75,2016-02-10 11:02:10.420,75,2016-02-11 13:49:21,697,4,1,1,1
127417,126973,NaPl126973,20_132_61 : parcelle de la zone de Paris,1945,1945,48.876212,2.409303,Parcelles situes au droit de l'ancien bastion ...,75,2016-02-10 11:02:10.420,75,2016-02-11 13:49:21,697,4,1,1,1
127418,126976,NaPl126976,20_132_64 : parcelle de la zone de Paris,1945,1945,48.87664,2.408964,Parcelles situes au droit de l'ancien bastion ...,75,2016-02-10 11:02:10.420,75,2016-02-11 13:49:21,697,4,1,1,1


In [8]:
named_places['fk_abob_type_napl'].unique()

<IntegerArray>
[697, 156, 696, 245, 698, 695]
Length: 6, dtype: Int32